In [1]:
# Load libraries

import pandas as pd
import pickle
import numpy as np

In [2]:
root="C:\\Users\\hmitt\\Downloads\\Team23_Project\\datasets\\"

In [3]:
df=pd.read_pickle(root+"corr_removed_feature_data.pkl")
pd.set_option('display.max_columns', None)
df

,user_id,product_id,total_orders_by_user,user_reorder_rate,add_to_cart_by_user_mean,days_since_prior_order_avg,product_last_bought_order,is_reorder_3,is_reorder_2,is_reorder_1,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered,prod_add_to_cart_order_mean,prod_order_total,prod_reorder_rate,is_organic,aisle_add_to_cart_order_mean,aisle_order_total,aisle_reorder_rate,aisle_0,aisle_1,aisle_2,aisle_3,aisle_4,aisle_5,aisle_6,aisle_7,dept_add_to_cart_order_mean,dept_unique_users,department_0,department_1,department_2,department_3,department_4,dow_mean,dow_std,doh_mean,doh_std,days_since_prior_order_mean,days_since_prior_order_std,products_by_user,reorder_rate_by_user,order_size_avg,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,10.0,0.900000,1.400000,17.593750,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0,3.721774,35791,0.776480,0,6.453676,357537,0.638832,0,0,1,1,0,1,1,1,6.976699,172795,0,0,0,1,1,2.644068,1.256194,10.542373,3.500355,18.546875,10.559065,59,0.694915,5.900000,6,6,9,0.666667,1.0,0.666667
1,1,10258,9.0,0.888889,3.333333,19.562500,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0,4.277492,1946,0.713772,0,9.319779,306487,0.519170,0,1,1,0,0,1,1,0,9.187743,174219,0,0,0,0,1,2.644068,1.256194,10.542373,3.500355,18.546875,10.559065,59,0.694915,5.900000,6,6,9,0.666667,1.0,0.666667
2,1,10326,1.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,0.0,11.0,4.0,8.0,14.0,0.0,4.191097,5526,0.652009,0,7.144228,3642188,0.718104,0,1,0,1,0,0,1,0,8.022875,193237,0,1,1,1,0,2.644068,1.256194,10.542373,3.500355,18.546875,10.559065,59,0.694915,5.900000,6,6,9,0.666667,1.0,0.666667
3,1,12427,10.0,0.900000,3.300000,17.593750,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,0.0,4.760037,6476,0.740735,0,8.464525,163524,0.591986,0,0,1,0,1,1,0,0,9.187743,174219,0,0,0,0,1,2.644068,1.256194,10.542373,3.500355,18.546875,10.559065,59,0.694915,5.900000,6,6,9,0.666667,1.0,0.666667
4,1,13032,3.0,0.666667,6.333333,21.671875,10.0,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0,5.622767,3751,0.657158,0,9.045796,377586,0.571584,0,0,0,1,1,0,1,0,9.124044,114169,0,1,1,0,0,2.644068,1.256194,10.542373,3.500355,18.546875,10.559065,59,0.694915,5.900000,6,6,9,0.666667,1.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,206209,43961,3.0,0.666667,8.000000,23.328125,12.0,1.0,0.0,0.0,14.0,6.0,14.0,30.0,0.0,9.194723,55371,0.630583,0,8.399527,1765313,0.638514,0,0,0,1,1,1,0,0,8.022875,193237,0,1,1,1,0,2.658915,1.538547,12.922481,2.737507,18.234375,10.919815,129,0.472868,9.923077,8,20,9,0.750000,0.7,0.444444
8474657,206209,44325,1.0,0.000000,8.000000,9.000000,7.0,0.0,0.0,0.0,14.0,6.0,14.0,30.0,0.0,10.109900,3485,0.401148,0,10.356121,266637,0.453062,0,0,0,1,1,1,1,0,10.212863,124820,0,1,0,1,0,2.658915,1.538547,12.922481,2.737507,18.234375,10.919815,129,0.472868,9.923077,8,20,9,0.750000,0.7,0.444444
8474658,206209,48370,1.0,0.000000,8.000000,30.000000,11.0,0.0,0.0,0.0,14.0,6.0,14.0,30.0,0.0,8.344942,3934,0.699288,0,7.825112,242996,0.528005,0,1,1,1,1,1,0,0,8.492163,111608,0,0,1,1,1,2.658915,1.538547,12.922481,2.737507,18.234375,10.919815,129,0.472868,9.923077,8,20,9,0.750000,0.7,0.444444
8474659,206209,48697,1.0,0.000000,6.000000,9.000000,7.0,0.0,0.0,0.0,14.0,6.0,14.0,30.0,0.0,8.763058,9783,0.357661,0,9.014857,245466,0.350843,0,0,1,1,1,0,1,1,9.593425,172755,0,0,0,1,0,2.658915,1.538547,12.922481,2.737507,18.234375,10.919815,129,0.472868,9.923077,8,20,9,0.750000,0.7,0.444444


In [4]:
def product_time(prior_data = None):
    
    """
    feature : how frequently product was reordered on any given hour ?
    """
    df = prior_data.groupby(['product_id','order_hour_of_day'])["reordered"].size()
    df = df/prior_data.groupby(["product_id"]).size()
    df = df.reset_index(name = 'hour_reorder_rate')
    return df

def product_day(prior_data = None):
    """
    feature: how frequently product was reordered on any given day ?
    """
    df = prior_data.groupby(['product_id','order_dow'])["reordered"].size()
    df = df/prior_data.groupby(["product_id"]).size()
    df = df.reset_index(name = 'day_reorder_rate')
    return df

def product_days_since_prior(prior_data = None):
    """
    feature: how frequently a product was reordered  given that 
            difference between 2 orders containing product in days ?
    """
    df = prior_data.groupby(['product_id','days_since_prior_order'])["reordered"].size()
    df = df/prior_data.groupby(["product_id"]).size()
    df = df.reset_index(name = 'p_days_since_prior_order_reorder_rate')
    return df

def user_days_since_prior(prior_data = None):
    """
    feature: how frequently user reordered any product given difference between 2 orders in days ?
    """
    df = prior_data.groupby(['user_id','days_since_prior_order'])["reordered"].size()
    df = df/prior_data.groupby(["user_id"]).size()
    df = df.reset_index(name = 'u_days_since_prior_order_reorder_rate')
    return df

def u_p_days_since_prior(prior_data = None):
    """
    feature: how frequently user reordered a particular product given difference between 2 orders in days ?
    """
    df = prior_data.groupby(["user_id","product_id","days_since_prior_order"])["reordered"].size()
    df = df/prior_data.groupby(["user_id","product_id"]).size()
    df = df.reset_index(name = 'days_since_prior_reorder_rate')
    return df

In [5]:
prior_order = pd.read_csv(root+'order_products__prior.csv')

In [6]:
aisles = pd.read_csv(root+'aisles.csv')
departments = pd.read_csv(root+'departments.csv')
train_order = pd.read_csv(root+'order_products__train.csv')
orders = pd.read_csv(root+'orders.csv')
products = pd.read_csv(root+'products.csv')

#imputation
orders.fillna(value = 0, inplace = True)

In [7]:
#Merge all files together
prior_orders_data = prior_order.sort_values(by=['order_id'])
prior_orders_data = pd.merge(left = prior_orders_data, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

col_order = ['user_id',
 'order_id',
 'product_id',
  'aisle_id',
 'department_id',
 'add_to_cart_order',
 'reordered',
 'product_name',
 'aisle',
 'department',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

prior_orders_data = prior_orders_data[col_order]
print("Shape of prior_orders_data :", prior_orders_data.shape)
prior_orders_data.head()

Shape of prior_orders_data : (32434489, 15)


,user_id,order_id,product_id,aisle_id,department_id,add_to_cart_order,reordered,product_name,aisle,department,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,202279,2,33120,86,16,1,1,Organic Egg Whites,eggs,dairy eggs,prior,3,5,9,8.0
1,202279,2,1819,88,13,8,1,All Natural No Stir Creamy Almond Butter,spreads,pantry,prior,3,5,9,8.0
2,202279,2,45918,19,13,4,1,Coconut Butter,oils vinegars,pantry,prior,3,5,9,8.0
3,202279,2,9327,104,13,3,0,Garlic Powder,spices seasonings,pantry,prior,3,5,9,8.0
4,202279,2,17794,83,4,6,1,Carrots,fresh vegetables,produce,prior,3,5,9,8.0


In [8]:
#Merge all files together

prior_orders_data = pd.concat([prior_order, train_order]).sort_values(by=['order_id'])
prior_orders_data = pd.merge(left = prior_orders_data, right = products,
                            left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = aisles,
                            left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = departments,
                        left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

prior_orders_data.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
2,1,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce,112108,train,4,4,10,9.0
3,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0
4,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0


In [9]:
#construct dataframe

hour_reorder_rate = product_time(prior_orders_data)
day_reorder_rate = product_day(prior_orders_data)
p_days_since_prior_order_reorder_rate = product_days_since_prior(prior_orders_data)
u_days_since_prior_order_reorder_rate = user_days_since_prior(prior_orders_data)
days_since_prior_reorder_rate = u_p_days_since_prior(prior_orders_data)

In [ ]:
#save these files

hour_reorder_rate.to_csv("hour_reorder_rate.csv", index = False)
day_reorder_rate.to_csv("day_reorder_rate.csv", index = False)
p_days_since_prior_order_reorder_rate.to_csv("p_days_since_prior_order_reorder_rate.csv", index = False)
u_days_since_prior_order_reorder_rate.to_csv("u_days_since_prior_order_reorder_rate.csv", index = False)
days_since_prior_reorder_rate.to_csv("days_since_prior_reorder_rate.csv", index = False)

In [ ]:
#save these files

hour_reorder_rate.to_pickle("hour_reorder_rate.pkl")
day_reorder_rate.to_pickle("day_reorder_rate.pkl")
#p_days_since_prior_order_reorder_rate.to_pickle("p_days_since_prior_order_reorder_rate.pkl")
#u_days_since_prior_order_reorder_rate.to_pickle("u_days_since_prior_order_reorder_rate.pkl")
days_since_prior_reorder_rate.to_pickle("days_since_prior_reorder_rate.pkl")

In [10]:
p_days_since_prior_order_reorder_rate['days_since_prior_order']=p_days_since_prior_order_reorder_rate['days_since_prior_order'].astype('int32')
p_days_since_prior_order_reorder_rate.dtypes

product_id                                 int64
days_since_prior_order                     int32
p_days_since_prior_order_reorder_rate    float64
dtype: object

In [11]:
u_days_since_prior_order_reorder_rate['days_since_prior_order']=u_days_since_prior_order_reorder_rate['days_since_prior_order'].astype('int32')
u_days_since_prior_order_reorder_rate.dtypes

user_id                                    int64
days_since_prior_order                     int32
u_days_since_prior_order_reorder_rate    float64
dtype: object

In [ ]:
p_days_since_prior_order_reorder_rate.to_pickle("p_days_since_prior_order_reorder_rate.pkl")
u_days_since_prior_order_reorder_rate.to_pickle("u_days_since_prior_order_reorder_rate.pkl")

In [12]:
product_mappings = pd.read_csv(root+"products.csv")

In [13]:
product_mappings = product_mappings[['product_id','product_name']]

In [ ]:
product_mappings.to_pickle("product_mappings.pkl")

In [14]:
user_last_purchase = pd.DataFrame(columns = ['user_id','date'])
user_last_purchase['user_id'] = orders['user_id'].unique()
user_last_purchase['date'] = '2023-01-19'

In [ ]:
user_last_purchase.to_pickle("user_last_purchase.pkl")

In [15]:
df = pd.merge(df, hour_reorder_rate, on=['product_id','order_hour_of_day'], how = 'left')
df[['hour_reorder_rate']]= df[['hour_reorder_rate']].fillna(value=0.0)

df = pd.merge(df, day_reorder_rate, on=['product_id','order_dow'], how = 'left')
df[['day_reorder_rate']]= df[['day_reorder_rate']].fillna(value=0.0)

df = pd.merge(df, p_days_since_prior_order_reorder_rate, on=['product_id','days_since_prior_order'], how = 'left')
df[['p_days_since_prior_order_reorder_rate']]= df[['p_days_since_prior_order_reorder_rate']].fillna(value=0.0)

df = pd.merge(df, u_days_since_prior_order_reorder_rate, on=['user_id','days_since_prior_order'], how = 'left')
df[['u_days_since_prior_order_reorder_rate']]= df[['u_days_since_prior_order_reorder_rate']].fillna(value=0.0)

df = pd.merge(df, days_since_prior_reorder_rate, on=["user_id","product_id",'days_since_prior_order'], how = 'left')
df[['days_since_prior_reorder_rate']]= df[['days_since_prior_reorder_rate']].fillna(value=0.0)

In [16]:
df.isnull().sum()

user_id                                  0
product_id                               0
total_orders_by_user                     0
user_reorder_rate                        0
add_to_cart_by_user_mean                 0
days_since_prior_order_avg               0
product_last_bought_order                0
is_reorder_3                             0
is_reorder_2                             0
is_reorder_1                             0
order_number                             0
order_dow                                0
order_hour_of_day                        0
days_since_prior_order                   0
reordered                                0
prod_add_to_cart_order_mean              0
prod_order_total                         0
prod_reorder_rate                        0
is_organic                               0
aisle_add_to_cart_order_mean             0
aisle_order_total                        0
aisle_reorder_rate                       0
aisle_0                                  0
aisle_1    

In [17]:
df['days_since_prior_order']=df['days_since_prior_order'].astype('int32')

In [19]:
df.shape

(8474661, 57)

In [20]:
df.to_pickle("Updated_final_dataset.pkl")

In [18]:
days_since_prior_reorder_rate

,user_id,product_id,days_since_prior_order,days_since_prior_reorder_rate
0,1,196,0.0,0.181818
1,1,196,14.0,0.181818
2,1,196,15.0,0.090909
3,1,196,19.0,0.090909
4,1,196,20.0,0.090909
...,...,...,...,...
27517679,206209,44325,9.0,1.000000
27517680,206209,48370,30.0,1.000000
27517681,206209,48697,9.0,1.000000
27517682,206209,48742,9.0,0.500000


In [ ]:
days_since_prior_reorder_rate['days_since_prior_order']=days_since_prior_reorder_rate['days_since_prior_order'].astype('int32')

In [ ]:
days_since_prior_reorder_rate.to_pickle("days_since_prior_reorder_rate.pkl")